<a href="https://colab.research.google.com/github/gabrielanatalia/PortfolioVisualizer/blob/main/Portfolio_Visualizer_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Portfolio Visualizer**



In [12]:
# @title 1. Load libraries and functions
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from datetime import date, timedelta
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.optimize import fsolve
import itertools
import plotly.graph_objects as go
import yfinance as yf
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
from matplotlib import colors
import statsmodels.formula.api as smf
from dateutil.parser import parse
import pytz
import re
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Javascript
from ipywidgets import VBox, HBox, Label
import time
from google.colab import widgets as gc_widgets
from contextlib import redirect_stdout
import os

!git clone https://github.com/gabrielanatalia/Projects/
%cd /content/Projects
import sys
sys.path.append('/content/Projects')
import port_cons as pc

import warnings
warnings.filterwarnings("ignore")

def convert_to_datetime(input_str, parserinfo=None):
    return parse(input_str, parserinfo=parserinfo)

TOLERANCE = 1e-10

def _allocation_risk(weights, covariances):

    portfolio_risk = np.sqrt((weights * covariances * weights.T))[0, 0]

    return portfolio_risk

def _assets_risk_contribution_to_allocation_risk(weights, covariances):

    portfolio_risk = _allocation_risk(weights, covariances)

    assets_risk_contribution = np.multiply(weights.T, covariances * weights.T) \
        / portfolio_risk

    return assets_risk_contribution

def _risk_budget_objective_error(weights, args):
    covariances = args[0]
    assets_risk_budget = args[1]
    weights = np.matrix(weights)

    portfolio_risk = _allocation_risk(weights, covariances)

    assets_risk_contribution = \
        _assets_risk_contribution_to_allocation_risk(weights, covariances)

    assets_risk_target = \
        np.asmatrix(np.multiply(portfolio_risk, assets_risk_budget))

    error = sum(np.absolute(assets_risk_contribution - assets_risk_target.T))[0, 0]
    return error

def _get_risk_parity_weights(covariances, assets_risk_budget, initial_weights):

    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},{'type': 'ineq', 'fun': lambda x: x})

    optimize_result = minimize(fun=_risk_budget_objective_error,
                               x0=initial_weights,
                               args=[covariances, assets_risk_budget],
                               method='SLSQP',
                               constraints=constraints,
                               tol=TOLERANCE,
                               options={'disp': False})

    weights = optimize_result.x
    print(optimize_result.message)
    return weights

def rebal_wgt_riskparity(returns_data, start_date, end_date, rebal_months=[4,10], halflife=3.5, annualized=252, shrink_covar=False):
    num_of_assets = len(returns_data.columns)

    ret_data_filtered = returns_data.loc[start_date:end_date]
    first_date = returns_data.index[0]
    tickers = list(returns_data.columns)

    weights = pd.DataFrame(0, index=ret_data_filtered.index, columns=ret_data_filtered.columns)
    weights.index = pd.to_datetime(weights.index, format='%Y-%m-%d')
    reb_flag = pd.DataFrame(0, index=weights.index, columns=['reb_flag'])

    alpha = 1 - math.exp(math.log(0.5) / (halflife * annualized))
    span = (2 / alpha) - 1

    for i in range(len(ret_data_filtered)):
        curr_date = ret_data_filtered.index[i]
        curr_date_format = curr_date.strftime('%Y-%m-%d')

        if (weights.index[i].month in rebal_months and weights.index[i-1].month != weights.index[i].month) or i==0:

            if shrink_covar:
                shrink_covar_matrix_model = LedoitWolf().fit(returns_data.loc[first_date:curr_date])
                covar_ann = pd.DataFrame(shrink_covar_matrix_model.covariance_ * annualized, index=tickers, columns=tickers)
                covar_ann.columns = tickers
                covar_ann.index = tickers
            else:
                exp_cov_matrix = returns_data.loc[first_date:curr_date].ewm(span=span).cov(pairwise=True).iloc[-num_of_assets:]
                covar_ann = exp_cov_matrix * annualized
                covar_ann.columns = tickers
                covar_ann.index = tickers

            valid_covar_ann = covar_ann.dropna(how='all')
            valid_covar_ann = valid_covar_ann.dropna(axis=1, how='all')
            valid_num_assets = len(valid_covar_ann)
            valid_tickers = valid_covar_ann.columns

            assets_risk_budget = np.ones([valid_num_assets]) / valid_num_assets
            initial_weights = np.ones([valid_num_assets]) / valid_num_assets

            # print(valid_covar_ann)
            new_wgt = _get_risk_parity_weights(valid_covar_ann.values, assets_risk_budget, initial_weights)
            new_wgt = pd.DataFrame(new_wgt).T
            new_wgt.columns = valid_tickers

            for ticker in valid_tickers:
                weights.at[curr_date_format, ticker] = new_wgt[ticker].values

            reb_flag.loc[curr_date_format] = True

        else:
            weights.iloc[i] = weights.iloc[i-1] * (1+ ret_data_filtered.iloc[i].fillna(0))
            weights_sum = weights.iloc[i].sum()
            weights.iloc[i] /= weights_sum
            reb_flag.iloc[i] = False

    weights = weights.rename(columns={c: c + '_wgt' for c in weights.columns})
    weights = pd.concat([reb_flag, weights], axis=1)

    return weights

def combine_backtest_data(portfolio_names):
    # combined portfolio returns
    all_port_ret = {}

    for portfolio in portfolio_names:
        perf_df = globals()[f"{portfolio}_perf"]
        ret_col = [col for col in perf_df.columns if col.endswith('_port_ret')]
        all_port_ret[portfolio] = perf_df[ret_col]

    df_all_port_ret = pd.concat(all_port_ret.values(), keys=all_port_ret.keys(), axis=1)
    df_all_port_ret.columns = [col[0] for col in df_all_port_ret.columns]

    # combined portfolio weights
    all_port_weights = {}

    for portfolio in portfolio_names:
        wgt_df = globals()[f"{portfolio}_wgt"]
        # wgt_col = [col for col in wgt_df.columns if col.endswith('_wgt')]
        # all_port_weights[portfolio] = wgt_df[wgt_col]
        all_port_weights[portfolio] = wgt_df

    df_all_port_weights = pd.concat(all_port_weights.values(), keys=all_port_weights.keys(), axis=1)

    # combined portfolio backtest daata
    all_port_bt = {}

    for portfolio in portfolio_names:
        bt_df = globals()[f"{portfolio}_perf"]
        all_port_bt[portfolio] = bt_df

    df_all_port_bt = pd.concat(all_port_bt.values(), keys=all_port_bt.keys(), axis=1)
    return df_all_port_ret, df_all_port_weights, df_all_port_bt


def print_arial(text):
    display(HTML(f"<div style='font-family: Arial, sans-serif'>{text}</div>"))

def print_arial_bold(text):
    display(HTML(f"<div style='font-family: Arial, sans-serif; font-weight: bold;'>{text}</div>"))

########################################################################################################################

# read asseet class, sector, country data from csv
data_path = '/content/Projects/Data/'
df_sec_list = pd.read_csv(data_path + 'PV_sec_list.csv')
df_sec_list['Remarks'] = df_sec_list['Remarks'].fillna('N/A')
df_asset_class = pd.read_csv(data_path + 'PV_asset_class.csv', index_col=0)
df_sector = pd.read_csv(data_path + 'PV_sector.csv', index_col=0)
df_country = pd.read_csv(data_path + 'PV_country.csv', index_col=0)

Cloning into 'Projects'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 205 (delta 27), reused 5 (delta 5), pack-reused 157 (from 1)
Receiving objects: 100% (205/205), 26.09 MiB | 9.88 MiB/s, done.
Resolving deltas: 100% (103/103), done.
/content/Projects


In [36]:
# @title 2. Input parameters

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

out = widgets.Output()

#####################################################################################################################
# Define a function to display securities for a given category
def display_securities_df(category):
    with out:
        out.clear_output(wait=True)  # Clear previous output
        filtered_df = df_sec_list[df_sec_list['Category'] == category][['Ticker', 'Name', 'Exchange','Remarks']].sort_values(by='Ticker', ascending=True)
        filtered_df = filtered_df.fillna('')
        display(filtered_df.reset_index(drop=True))

# Create buttons for each category
syfeportfolio_button = widgets.Button(description="Syfe portfolio")
equity_button = widgets.Button(description="Equities")
fixed_income_button = widgets.Button(description="Fixed income")
commodities_button = widgets.Button(description="Commodities")
multi_asset_button = widgets.Button(description="Multi-asset")
index_button = widgets.Button(description="Index")

# Assign the display_securities_df function to each button's on_click event
syfeportfolio_button.on_click(lambda b: display_securities_df("Syfe portfolio"))
equity_button.on_click(lambda b: display_securities_df("Equities"))
fixed_income_button.on_click(lambda b: display_securities_df("Fixed income"))
commodities_button.on_click(lambda b: display_securities_df("Commodities"))
multi_asset_button.on_click(lambda b: display_securities_df("Multi-asset"))
index_button.on_click(lambda b: display_securities_df("Index"))

# Display the buttons and output widget
print_arial_bold('See full list of securities')
display(widgets.HBox([syfeportfolio_button, equity_button, fixed_income_button, commodities_button, multi_asset_button,index_button]))
display(out)

#####################################################################################################################
# @title Alternatively, check if a security is in the list

etf_ticker_input = widgets.Text(
    value='',
    placeholder='Enter ETF ticker',
    description='ETF Ticker:',
    disabled=False
)

# Create an output widget to display the result
output_check = widgets.Output()

# Define a function to check if the ETF ticker is in df_sec_list
def check_security(b):
  with output_check:
    clear_output(wait=True)
    ticker = etf_ticker_input.value.upper()  # Convert to uppercase for case-insensitivity
    if ticker in df_sec_list['Ticker'].values:
      print(f"{ticker} is in the list.")
    else:
      print(f"{ticker} is not in the list.")

# Create a button to trigger the check
print('\n')
print_arial_bold('Alternatively, check if a security is in the list')
check_button = widgets.Button(description="Check Security")
check_button.on_click(check_security)

# Display the widgets
display(etf_ticker_input, check_button, output_check)

#####################################################################################################################

bbg_last_update_date = pd.read_csv(data_path + 'PV_daily_ret.csv', index_col=0, usecols=[0], parse_dates=['Date'],dayfirst=True).index[-1]
bbg_last_update_date = bbg_last_update_date.strftime('%Y-%m-%d')

print('\n')
print_arial_bold('Choose data source')
print_arial('• Bloomberg: Provides total return data, but not updated in real-time. Last updated: ' + str(bbg_last_update_date))
print_arial('• Yahoo Finance: Provides live data (as of last close), but includes only price returns.')

data_source_widget = widgets.RadioButtons(
       options=['Bloomberg', 'Yahoo Finance'],
       description='Data Source:',
       disabled=False)
display(data_source_widget)

#####################################################################################################################
# @title Input data parameters

# Widget for start date
start_date_widget = widgets.DatePicker(
    description='Start Date:',
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Widget for end date
end_date_widget = widgets.DatePicker(
    description='End Date:',
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Widget for benchmark tickers
bm_tickers_widget = widgets.Text(
    value='',
    description='Benchmark Tickers:',
    disabled=False,
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Widget for backtest offset days
backtest_offset_days_widget = widgets.IntText(
    value=365,
    description='Observation window (Days):',
    disabled=False,
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Display the widgets
print_arial_bold('Input data parameters')
print_arial("<br>".join([
    "•   Start date: format - DD/MM/YYYY",
    "•   End date: format - DD/MM/YYYY",
    "•   Benchmark ticker format - ETF1, ETF2, ETF3, ... | e.g. ACWI, URTH, SPY",
    "•   Observation window (days): the number of days after the specified start date used to initialize the optimization process - \
    this ensures enough data is available before the backtest begins (i.e. if lookback window is 365 days and start date is Jan 1 2010, the actual backtest will begin in Jan 1 2011)"]))

print('\n')
print_arial("❗ If no start and end date is specified, the backtest will begin from the earliest common inception until latest available data")

display(start_date_widget)
display(end_date_widget)
display(bm_tickers_widget)
display(backtest_offset_days_widget)


# Function to update variables based on widget values
def update_variables(change):
    global start_date, end_date, tickers, bm_tickers, all_tickers, backtest_offset_days

    start_date = start_date_widget.value

    # Check if end_date_widget has a value (user input)
    if end_date_widget.value is None:
        # If not, set end_date to the latest date from ret_all (assuming ret_all is the downloaded DataFrame)
        if 'ret_all' in globals() and not ret_all.empty:  # Check if ret_all exists and is not empty
            end_date = ret_all.index.max().date()
        else:
            end_date = None  # Or handle the case where ret_all is not yet available
    else:
        end_date = end_date_widget.value  # Use user-provided end date

    tickers = re.split(r'\s*,\s*', tickers_widget.value)
    bm_tickers = re.split(r'\s*,\s*', bm_tickers_widget.value)

    all_tickers = list(set(tickers + bm_tickers))

    backtest_offset_days = backtest_offset_days_widget.value

# Observe widget changes and update variables
start_date_widget.observe(update_variables, names='value')
end_date_widget.observe(update_variables, names='value')
bm_tickers_widget.observe(update_variables, names='value')
backtest_offset_days_widget.observe(update_variables, names='value')

# Initial variable update
update_variables(None)

def modify_tickers_based_on_exchange(all_tickers):
  modified_tickers = []
  for ticker in all_tickers:
      exchange = df_sec_list.loc[df_sec_list['Ticker'] == ticker, 'Exchange'].iloc[0] if ticker in df_sec_list['Ticker'].values else None
      if exchange == 'London':
          modified_tickers.append(ticker + '.L')
      elif exchange == 'Swiss':
          modified_tickers.append(ticker + '.SW')
      else:
          modified_tickers.append(ticker)
  return modified_tickers

def download_data(data_source, start_date, end_date):
  global ret_all, ret_bm, earliest_start_date, all_tickers # Declare as global
  all_tickers = get_unique_tickers()

  if data_source == 'Yahoo Finance':
    modified_tickers = modify_tickers_based_on_exchange(all_tickers)

    if start_date is None and end_date is None:
      df_all = yf.download(modified_tickers, period='max')['Adj Close']
    else:
      df_all = yf.download(modified_tickers, start=start_date, end=end_date)['Adj Close']

    df_all.index = df_all.index.tz_localize(None)
    ret_all = df_all.pct_change().dropna()
    ret_all.columns = all_tickers
    ret_bm = ret_all[bm_tickers]


  elif data_source == 'Bloomberg':
    ret_all = pd.read_csv(data_path + 'PV_daily_ret.csv',header=0, index_col='Date', parse_dates=['Date'],dayfirst=True)[all_tickers]/100
    # if start_date is None and end_date is None:
    if start_date is None:
      ret_all = ret_all = ret_all.dropna()
    else:
      start_date = start_date.strftime('%Y-%m-%d')
      end_date = end_date.strftime('%Y-%m-%d')
      ret_all = ret_all.loc[start_date:end_date].dropna()

    ret_bm = ret_all[bm_tickers]

  earliest_start_date = ret_all.index.min()
  start_date_timestamp = pd.Timestamp(start_date)

  # Compare earliest_start_date with start_date_timestamp
  if earliest_start_date > start_date_timestamp:
    print('\n', 'Earliest start date: ', earliest_start_date)
  else:
    pass

download_button = widgets.Button(description="Download Data")

#####################################################################################################################
# @title Input portfolio parameters
from ipywidgets import widgets, VBox, HTML

# Function to create the fields for a single portfolio

def create_portfolio_widgets(portfolio_number):
    """Creates and returns widgets for a single portfolio."""
    widgets_dict = {
        'header': widgets.HTML(f"<h3>Portfolio {portfolio_number}</h3>"),
        'tickers': widgets.Text(value='', description='Tickers:'),
        'allocation_type': widgets.Dropdown(
            options=['Max sharpe', 'Min vol', 'Risk parity', 'Equal weight', 'Custom weight'],
            value='Max sharpe',
            description='Allocation:'
        ),
        'name': widgets.Text(value=f'P{portfolio_number}', description='Name:'),
        'rebal_freq': widgets.Dropdown(
            options=['Monthly', 'Quarterly', 'Semi-annually', 'Annually'],
            value='Annually',
            description='Rebal Freq:'
        ),
        'min_weight': widgets.FloatText(value=0.0, description='Min Weight:'),
        'max_weight': widgets.FloatText(value=1.0, description='Max Weight:'),
        'custom_weight': widgets.Text(
            value='',
            description='Input Weight:',
            layout=widgets.Layout(visibility='hidden')  # Initially hidden
        ),
    }

    def toggle_custom_weight_visibility(change):
        widgets_dict['custom_weight'].layout.visibility = (
            'visible' if widgets_dict['allocation_type'].value == 'Custom weight' else 'hidden'
        )

    widgets_dict['allocation_type'].observe(toggle_custom_weight_visibility, names='value')
    return widgets_dict

# Function to display portfolio widgets
def display_portfolio_widgets(widgets_dict):
    """Organizes portfolio widgets into a VBox for display."""
    return VBox(list(widgets_dict.values()))

# Initialize portfolio storage
portfolio_widgets = {}
portfolio_count = 0
portfolio_output = widgets.Output()

# Add and Delete Portfolio Buttons
add_portfolio_button = widgets.Button(description="Add Portfolio")
delete_portfolio_button = widgets.Button(description="Delete Portfolio")

def add_portfolio(b):
    global portfolio_count
    if portfolio_count < 5:  # Limit to 5 portfolios
        portfolio_count += 1
        portfolio_widgets[portfolio_count] = create_portfolio_widgets(portfolio_count)
        with portfolio_output:
            display(display_portfolio_widgets(portfolio_widgets[portfolio_count]))
    else:
        print("Maximum number of portfolios reached.")

def delete_portfolio(b):
    global portfolio_count
    if portfolio_count > 1:
        with portfolio_output:
            clear_output(wait=True)
            del portfolio_widgets[portfolio_count]
            portfolio_count -= 1
            for i in range(1, portfolio_count + 1):
                display(display_portfolio_widgets(portfolio_widgets[i]))
    else:
        print("Cannot delete the last remaining portfolio.")

add_portfolio_button.on_click(add_portfolio)
delete_portfolio_button.on_click(delete_portfolio)

# Search Functionality
print('\n')
print_arial_bold('Search and add securities to your portfolio or type in the desired securities in the "Tickers" field')

search_input = widgets.Text(value='', placeholder='Search for a security/portfolio', description='Search:', disabled=False)
search_output = widgets.Output()

def add_ticker_to_portfolio(ticker, portfolio_number):
    """Add a ticker to a specific portfolio."""
    portfolio_tickers = portfolio_widgets[portfolio_number]['tickers']
    current_tickers = portfolio_tickers.value
    if current_tickers:
        portfolio_tickers.value = current_tickers + ", " + ticker
    else:
        portfolio_tickers.value = ticker

def on_search_input_change(change):
    """Filter and display the search results."""
    with search_output:
        clear_output(wait=True)
        search_term = search_input.value
        if search_term:
            filtered_df = df_sec_list[df_sec_list['Ticker'].str.contains(search_term, case=False, na=False) |
                                      df_sec_list['Name'].str.contains(search_term, case=False, na=False)]
        else:
            filtered_df = df_sec_list

        if not filtered_df.empty:
            rows = []
            for _, row in filtered_df.iterrows():
                add_buttons = []
                for i in range(1, portfolio_count + 1):
                    button = widgets.Button(description=f"Add to Portfolio {i}", button_style='primary')
                    button.on_click(lambda b, t=row['Ticker'], p=i: add_ticker_to_portfolio(t, p))
                    add_buttons.append(button)

                rows.append(HBox([
                    widgets.Label(row['Ticker'], layout=widgets.Layout(width='150px')),
                    widgets.Label(row['Name'], layout=widgets.Layout(width='300px')),
                    HBox(add_buttons)
                ]))
            display(VBox(rows))
        else:
            display(widgets.Label("No matching results found."))

search_input.observe(on_search_input_change, names='value')

# Display Widgets
display(search_input, search_output, portfolio_output, HBox([add_portfolio_button, delete_portfolio_button]))

# Preload Portfolio 1
portfolio_count += 1
portfolio_widgets[portfolio_count] = create_portfolio_widgets(portfolio_count)
with portfolio_output:
    display(display_portfolio_widgets(portfolio_widgets[portfolio_count]))

#####################################################################################################################
# @title Confirm portfolio parameters
def get_portfolio_specs():
    portfolio_specs = {}
    num_portfolios = len(portfolio_widgets)
    for i in range(1, num_portfolios + 1):
        portfolio_specs[i] = {
            'tickers': re.split(r'\s*,\s*', portfolio_widgets[i]['tickers'].value),
            'allocation_type': portfolio_widgets[i]['allocation_type'].value,
            'name': portfolio_widgets[i]['name'].value,
            'min_weight': float(portfolio_widgets[i]['min_weight'].value),
            'max_weight': float(portfolio_widgets[i]['max_weight'].value),
            'custom_weight': portfolio_widgets[i]['custom_weight'].value,  # Get the value of the custom weight widget
            'rebal_freq': portfolio_widgets[i]['rebal_freq'].value
        }

        if portfolio_specs[i]['allocation_type'] == 'Custom weight':
            try:
                portfolio_specs[i]['custom_weight'] = [float(x) for x in portfolio_specs[i]['custom_weight'].split(',')]
            except ValueError:
                print("Invalid custom weight input. Please enter comma-separated values and ensure values sum to 1 (e.g 0.3,0.3,0.4)")
                # You might want to handle the error more gracefully here.

        if portfolio_specs[i]['rebal_freq'] == 'Annually':
            portfolio_specs[i]['rebal_months'] = [1]
        elif portfolio_specs[i]['rebal_freq'] == 'Semi-annually':
            portfolio_specs[i]['rebal_months'] = [1, 7]
        elif portfolio_specs[i]['rebal_freq'] == 'Quarterly':
            portfolio_specs[i]['rebal_months'] = [1, 4, 7, 10]
        elif portfolio_specs[i]['rebal_freq'] == 'Monthly':
            portfolio_specs[i]['rebal_months'] = list(range(1, 13))

    return portfolio_specs


def get_unique_tickers():
        all_tickers_set = set(bm_tickers)  # Start with benchmark tickers
        for portfolio_widgets_dict in portfolio_widgets.values():
            portfolio_tickers = re.split(r'\s*,\s*', portfolio_widgets_dict['tickers'].value)
            all_tickers_set.update(portfolio_tickers)  # Add portfolio tickers
        return list(all_tickers_set)

# Function to execute on button click
def on_download_data_and_confirm_param_button_clicked(b):
      global start_date, end_date, bm_tickers, all_tickers, backtest_offset_days, data_source_widget, earliest_start_date, portfolio_specs, df_portfolio_specs, all_port_ret, all_port_wgt, all_port_bt, ret_bm_filtered, constituent_summary, yearly_df, perf_summary, port_yearly_ret, relative_perf, df_ports_alloc_latest, df_ports_alloc_avg, port_turnover_df, df_forecast

      download_data(data_source_widget.value, start_date, end_date)

      portfolio_specs = get_portfolio_specs()
      df_portfolio_specs = pd.DataFrame(portfolio_specs).T

      # Get the unique tickers after the portfolio specifications are confirmed
      all_tickers = get_unique_tickers()
      tickers = [ticker for ticker in all_tickers if ticker not in bm_tickers]

      display(df_portfolio_specs)
      print('\n')
      check_custom_weight_sum(portfolio_specs)


# Define constraints function
def create_constraints(min_weight, max_weight):
    return ({
        'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1
    }, {
        'type': 'ineq', 'fun': lambda weights: weights - min_weight
    }, {
        'type': 'ineq', 'fun': lambda weights: max_weight - weights
    })


def create_custom_weight_dict(portfolio_specs, portfolio_number):
  # Creates a dictionary with ticker as key and custom weight as value.
  portfolio = portfolio_specs[portfolio_number]
  tickers = portfolio['tickers']
  custom_weights = portfolio['custom_weight']

  if len(tickers) != len(custom_weights):
    raise ValueError("Number of tickers and custom weights must be equal.")

  custom_weight_dict = dict(zip(tickers, custom_weights))
  return custom_weight_dict

def check_custom_weight_sum(portfolio_specs):

  for portfolio_number, portfolio_data in portfolio_specs.items():
      if portfolio_data['allocation_type'] == 'Custom weight':
          custom_weights = portfolio_data['custom_weight']
          if np.isclose(np.sum(custom_weights), 1.0):
              print("Portfolio specifications confirmed! ✅")
          else:
              print(f"Portfolio {portfolio_number}: Custom weights do not sum to 1. Please amend.")


#####################################################################################################################

# Create a confirmation button
confirm_button = widgets.Button(description="Confirm Data and Portfolio Specs", layout=widgets.Layout(width='auto'))

# Attach the function to the button's on_click event
confirm_button.on_click(on_download_data_and_confirm_param_button_clicked)

# Display the button
print('\n')
print_arial_bold('Confirm data and portfolio parameters')
display(confirm_button)

#####################################################################################################################
# @title Run optimisation

optimization_output = widgets.Output()  # Create an output widget

def run_optimization():

    num_portfolios = len(portfolio_widgets)
    portfolio_specs = get_portfolio_specs()
    progress_bar = widgets.IntProgress(
        value=0,
        min=0,
        max=5,  # Total number of portfolios
        description='Optimising:',
        bar_style='',
        style={'bar_color': 'navy'},
        orientation='horizontal'
    )
    # Display the progress bar
    with optimization_output:
      display(progress_bar)
    with redirect_stdout(open(os.devnull, 'w')):
      for i in range(1, num_portfolios + 1):
          portfolio = portfolio_specs[i]
          portfolio_name = portfolio['name']
          tickers = portfolio['tickers']
          allocation_type = portfolio['allocation_type']
          min_weight = portfolio['min_weight']
          max_weight = portfolio['max_weight']
          custom_weight = portfolio['max_weight']  # or portfolio['custom_weight'] if intended
          rebal_months = portfolio['rebal_months']

          # Create constraints for each portfolio based on the min and max weight
          constraints = create_constraints(min_weight, max_weight)

          # Calculate weights based on allocation type:
          if allocation_type == 'Max sharpe':
              globals()[f"{portfolio_name}_wgt"] = pc.max_sharpe_rebal_wgt(
                  ret_all[tickers],
                  start_date=earliest_start_date + timedelta(days=backtest_offset_days),
                  end_date=end_date,
                  constraints_dict=constraints,
                  shrink_covar=False,
                  rebal_months=rebal_months,
                  annualized=261,
                  halflife=3.5
              )
          elif allocation_type == 'Min vol':
              globals()[f"{portfolio_name}_wgt"] = pc.min_vol_rebal_wgt(
                  ret_all[tickers],
                  start_date=earliest_start_date + timedelta(days=backtest_offset_days),
                  end_date=end_date,
                  constraints_dict=constraints,
                  shrink_covar=False,
                  rebal_months=rebal_months,
                  annualized=261,
                  halflife=3.5
              )
          elif allocation_type == 'Risk parity':
              globals()[f"{portfolio_name}_wgt"] = rebal_wgt_riskparity(
                  ret_all[tickers],
                  start_date=earliest_start_date + timedelta(days=backtest_offset_days),
                  end_date=end_date,
                  rebal_months=rebal_months,
                  halflife=3.5,
                  annualized=261,
                  shrink_covar=False
              )

          elif allocation_type == 'Equal weight':
              globals()[f"{portfolio_name}_wgt"] = pc.equal_weight_rebal_wgt(
                  ret_all[tickers],
                  start_date=earliest_start_date + timedelta(days=backtest_offset_days),
                  end_date=end_date,
                  rebal_months=rebal_months,
              )

          elif allocation_type == 'Custom weight':
              globals()[f"{portfolio_name}_wgt"] = pc.fixed_weight_rebal_wgt(
                ret_all[tickers],
                start_date=earliest_start_date + timedelta(days=backtest_offset_days),
                end_date=end_date,
                fixed_weights=create_custom_weight_dict(portfolio_specs, i),
                rebal_months=[1],
          )
          # Calculate performance (indented correctly):
          globals()[f"{portfolio_name}_perf"] = pc.calc_port_perf(ret_all.loc[earliest_start_date + timedelta(days=backtest_offset_days):end_date][tickers],
              globals()[f"{portfolio_name}_wgt"],
              name_prefix=portfolio_name
          )

          # Update the progress bar
          progress_bar.value += 1

    # Display "Optimisation complete!" after the loop is finished
    optimization_output.clear_output(wait=False)  # Clear the output widget after completion
    print("Optimisation complete!")

# Create the button and link it to the function
print('\n')
print_arial_bold('Run optimisation')

run_optimisation_button = widgets.Button(description="Run Optimisation")
run_optimisation_button.on_click(lambda b: run_optimization())  # Call the function on click
display(run_optimisation_button)
display(optimization_output)


HTML(value="<div style='font-family: Arial, sans-serif; font-weight: bold;'>See full list of securities</div>"…

Output()

HTML(value="<div style='font-family: Arial, sans-serif; font-weight: bold;'>Alternatively, check if a security…

Text(value='', description='ETF Ticker:', placeholder='Enter ETF ticker')

Button(description='Check Security', style=ButtonStyle())

...

HTML(value="<div style='font-family: Arial, sans-serif; font-weight: bold;'>Confirm data and portfolio paramet…

Button(description='Confirm Data and Portfolio Specs', layout=Layout(width='auto'), style=ButtonStyle())

HTML(value="<div style='font-family: Arial, sans-serif; font-weight: bold;'>Run optimisation</div>")

Button(description='Run Optimisation', style=ButtonStyle())

Output()

# Output

In [29]:
# @title 3. Historical weights at each point of rebalance

for i in range(1, len(portfolio_widgets) + 1):
    portfolio = portfolio_specs[i]
    portfolio_name = portfolio['name']
    print(portfolio_name)
    pc.display_rebal_wgt(globals()[f"{portfolio_name}_wgt"])
    print('\n')

P1


,reb_flag,Core E100_wgt,Thematic Disruptive Tech_wgt
Date,,,
2021-01-04 00:00:00,100.00%,50.97%,49.03%
2022-01-03 00:00:00,100.00%,55.73%,44.27%
2023-01-03 00:00:00,100.00%,59.95%,40.05%
2024-01-02 00:00:00,100.00%,60.63%,39.37%




P2


,reb_flag,Core Growth_wgt,Thematic ESG & Clean Energy_wgt
Date,,,
2021-01-04 00:00:00,100.00%,50.00%,50.00%
2022-01-03 00:00:00,100.00%,50.00%,50.00%
2023-01-03 00:00:00,100.00%,50.00%,50.00%
2024-01-02 00:00:00,100.00%,50.00%,50.00%


# Key Data

In [30]:
# @title 4. Generate constituent key metrics

# perf summary
constituent_summary = pc.performance_summary_constituents(ret_all, start_date=start_date, end_date=end_date, frequency='daily')
# calendar year returns
yearly_df = pc.constituents_calendar_year_returns(ret_all, frequency='daily')

def generate_constituents_key_metrics_func(b):
    clear_output(wait=True)  # Clear previous output
    generate_constituents_key_metrics()

def generate_constituents_key_metrics():
    # Define tab names
    tab_names = ["Key Summary", "Calendar Year Returns", "Monthly Returns", "Cumulative Returns", "Return/Risk", "Returns Distribution", "Volatility", "Drawdown", "Correlation"]

    # Create TabBar
    t = gc_widgets.TabBar(tab_names)

    # Populate tabs with content
    with t.output_to(tab_names[0]):  # Key Summary
        display(constituent_summary.T)

    with t.output_to(tab_names[1]):  # Calendar Year Returns
        print('Date range: ', ret_all.index.min(), ' - ', ret_all.index.max(), '\n')
        display(yearly_df.pipe(pc.apply_style_heatmap_ret))

    with t.output_to(tab_names[2]):  # Monthly Returns
      for i in ret_all.columns:
        security_returns = ret_all[[i]]
        monthly_performance = pc.monthly_performance_table(security_returns)
        print(i)
        display(monthly_performance.pipe(pc.apply_style_heatmap_ret))
        print('\n')

    with t.output_to(tab_names[3]):  # Cumulative Returns
        display(pc.plot_cumulative_returns(ret_all, show_data=True))

    with t.output_to(tab_names[4]):  # Return/risk scatter plot
        display(pc.plot_return_risk_scatter_maxrange(ret_all))

    with t.output_to(tab_names[5]):  # Returns Distribution
        pc.plot_returns_distribution_boxplot(ret_all)

    with t.output_to(tab_names[6]):  # Rolling volatility
        pc.plot_rolling_volatility(ret_all, window=261, title='Rolling 1Y Volatility')

    with t.output_to(tab_names[7]):  # Drawdown
        display(pc.plot_drawdowns(ret_all, show_data=True))

    with t.output_to(tab_names[8]):  # Correlation
        pc.plot_correlation_heatmap(ret_all)

    display(t)

# print('\n')
# print_arial_bold('Generate Constituents Key Metrics')
# constituents_key_metrics_button = widgets.Button(description="Generate Constituents Key Metrics", layout=widgets.Layout(width='auto'))
# constituents_key_metrics_button.on_click(generate_constituents_key_metrics_func)
# display(constituents_key_metrics_button)
generate_constituents_key_metrics()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
# @title 5. Generate portfolio key metrics

########################################################################################################################

port_names = []  # Initialize an empty list to store names

# Iterate through the portfolio specifications
for portfolio_number, portfolio_data in portfolio_specs.items():
    # Extract the 'name' attribute and append it to the list
    port_names.append(portfolio_data['name'])

all_port_ret, all_port_wgt, all_port_bt = combine_backtest_data(port_names)
ret_bm_filtered = ret_bm.loc[earliest_start_date + timedelta(days=backtest_offset_days):]
ret_bm_filtered.iloc[0] = 0
all_port_ret = pd.concat([all_port_ret, ret_bm], axis=1).dropna()

########################################################################################################################

# perf summary
perf_summary = pc.performance_summary(all_port_ret[port_names], all_port_wgt, all_port_ret[bm_tickers[0]], benchmark_returns=ret_bm,
                                    start_date=earliest_start_date + timedelta(days=backtest_offset_days), end_date=end_date, frequency='daily', rebal_per_year=1)

# calendar year returns
port_yearly_ret = pc.constituents_calendar_year_returns(all_port_ret, frequency='daily')
relative_perf = pc.calculate_relative_per(all_port_ret[port_names], all_port_ret[bm_tickers[0]])
# latest and average allocation
df_ports_alloc_latest = pc.compare_port_alloc(all_port_wgt, port_names, latest_data=True)
df_ports_alloc_avg = pc.compare_port_alloc(all_port_wgt, port_names, latest_data=False)

# portfolio turnover
port_turnover_df = pd.DataFrame()

for portfolio in port_names:
    turnover = pc.portfolio_turnover(all_port_wgt[portfolio])
    port_turnover_df = pd.concat([port_turnover_df, turnover],axis=1)

port_turnover_df.columns = port_names

df_forecast = pc.forecast_portfolio(all_port_ret)

# def generate_key_metrics_func(b):
#     clear_output(wait=True)  # Clear previous output
#     generate_key_metrics()

def generate_key_metrics():
    # Define tab names
    tab_names = ["Key Summary", "Calendar Year Returns", "Monthly Returns", "Cumulative Returns", "Return/Risk", "Returns Distribution", "Volatility", "Portfolio Drawdown", "Correlation", "Forecast Simulation"]

    # Create TabBar
    t = gc_widgets.TabBar(tab_names)

    # Populate tabs with content
    with t.output_to(tab_names[0]):  # Key Summary
        display(perf_summary)

    with t.output_to(tab_names[1]):  # Calendar Year Returns
        print('Calendar year returns')
        display(port_yearly_ret.pipe(pc.apply_style_heatmap_ret))
        print('\n')
        print('Relative calendar year returns')
        display(pc.apply_style_heatmap_ret(relative_perf, subset=relative_perf.columns[1:]))

    with t.output_to(tab_names[2]):  # Monthly Returns
      for port in port_names:
        portfolio_returns = all_port_ret[[port]]
        monthly_performance = pc.monthly_performance_table(portfolio_returns)
        print(port)
        display(monthly_performance.pipe(pc.apply_style_heatmap_ret))
        print('\n')

    with t.output_to(tab_names[3]):  # Cumulative Returns
        display(pc.plot_cumulative_returns(all_port_ret, show_data=True))

    with t.output_to(tab_names[4]):  # Return/risk scatter plot
        display(pc.plot_return_risk_scatter_maxrange(all_port_ret))

    with t.output_to(tab_names[5]):  # Returns Distribution
        pc.plot_returns_distribution_boxplot(all_port_ret)

    with t.output_to(tab_names[6]):  # Rolling volatility
        pc.plot_rolling_volatility(all_port_ret, window=261, title='Rolling 1Y Volatility')

    with t.output_to(tab_names[7]):  # Portfolio Drawdown
        display(pc.plot_drawdowns(all_port_ret, show_data=True))

    with t.output_to(tab_names[8]):  # Correlation
        pc.plot_correlation_heatmap(all_port_ret)
        pc.plot_rolling_correlation(all_port_ret[port_names], benchmark=all_port_ret[bm_tickers[0]], window=261, title='Rolling 1Y Correlation')

    with t.output_to(tab_names[9]):
        for i in port_names:
          pc.plot_forecast_simulation(df_forecast.filter(like=i))

    display(t)

# key_metrics_button = widgets.Button(description="Generate Key Metrics", layout=widgets.Layout(width='auto'))
# key_metrics_button.on_click(generate_key_metrics_func)
# display(key_metrics_button)
generate_key_metrics()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
# @title 6. Generate key holdings & allocations

# asset class exposure
assetclass_analysis_list = [pc.exposure_analysis(all_port_wgt[portfolio], df_asset_class, latest_data=True).rename(columns={0: portfolio}) for portfolio in port_names]
assetclass_breakdown_all = pd.concat(assetclass_analysis_list, axis=1)
assetclass_breakdown_all.columns = port_names
assetclass_breakdown_all = pd.concat([df_asset_class[bm_tickers], assetclass_breakdown_all], axis=1)
assetclass_breakdown_all = (assetclass_breakdown_all.loc[(assetclass_breakdown_all != 0).any(axis=1)].sort_values(by=assetclass_breakdown_all.columns[0], ascending=False))
# country exposure
country_analysis_list = [pc.exposure_analysis(all_port_wgt[portfolio], df_country, latest_data=True).rename(columns={0: portfolio}) for portfolio in port_names]
country_breakdown_all = pd.concat(country_analysis_list, axis=1)
country_breakdown_all.columns = port_names
country_breakdown_all = pd.concat([df_country[bm_tickers], country_breakdown_all], axis=1)
country_breakdown_all = (country_breakdown_all.loc[(country_breakdown_all != 0).any(axis=1)].sort_values(by=country_breakdown_all.columns[0], ascending=False))
country_breakdown_all.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage)
# sector exposure
sector_analysis_list = [pc.exposure_analysis(all_port_wgt[portfolio], df_sector, latest_data=True).rename(columns={0: portfolio}) for portfolio in port_names]
sector_breakdown_all = pd.concat(sector_analysis_list, axis=1)
sector_breakdown_all.columns = port_names
sector_breakdown_all = pd.concat([df_sector[bm_tickers], sector_breakdown_all], axis=1)
sector_breakdown_all = (sector_breakdown_all.loc[(sector_breakdown_all != 0).any(axis=1)].sort_values(by=sector_breakdown_all.columns[0], ascending=False))
sector_breakdown_all.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage)

# def generate_key_holdings_func(b):
#     clear_output(wait=True)  # Clear previous output
#     generate_key_holdings()

def generate_key_holdings():
    # Define tab names
    tab_names = ["Latest Allocation", "Average Allocation", "Historical Allocation", "Asset class exposure", "Country exposure", "Sector exposure"]

    # Create TabBar
    t = gc_widgets.TabBar(tab_names)

    # Populate tabs with content
    with t.output_to(tab_names[0]):  # Latest Allocation
        display(df_ports_alloc_latest.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage))

    with t.output_to(tab_names[1]):  # Average Allocation
        display(df_ports_alloc_avg.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage))

    with t.output_to(tab_names[2]):  # Portfolio Allocation
      for port in port_names:
        print(port)
        pc.plot_weights_and_turnover(all_port_wgt[port], show_data=True, show_turnover=False, show_rebal=True)

    with t.output_to(tab_names[3]):  # Asset class exposure
        display(assetclass_breakdown_all.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage))

    with t.output_to(tab_names[4]):  # Country exposure
        display(country_breakdown_all.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage))

    with t.output_to(tab_names[5]):  # Sector xposure
        display(sector_breakdown_all.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage))

    display(t)

# key_holdings_button = widgets.Button(description="Generate Key Holdings & Allocations", layout=widgets.Layout(width='auto'))
# key_holdings_button.on_click(generate_key_holdings_func)
# display(key_holdings_button)
generate_key_holdings()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,ACWI,P1,P2
Sector,,,
Information Technology,24.83%,40.06%,16.74%
Financials,15.83%,9.74%,10.36%
Health Care,11.36%,7.26%,6.02%
Industrials,10.43%,11.52%,20.40%
Consumer Discretionary,10.04%,9.68%,9.11%
Communication Services,7.73%,9.84%,5.16%
Consumer Staples,6.42%,4.27%,3.96%
Energy,4.47%,1.95%,2.00%
Materials,4.01%,2.47%,4.08%


<IPython.core.display.Javascript object>

# Factor exposure (Fama French 5 factors)

1.   MKT: Market risk premium - the excess return of the market over risk free rate
2.   SMB: Size (small minus big) - captures the size effect, where smaller firms tend to outperform larger firms
3. HML: Value (high minus low) - reflects the difference in returns between stocks with high book-to-market ratios (value stocks) and those with low book-to-market ratios (growth stocks)
4. RMW: Profitability/quality (robust minus weak) - measures the difference in returns between firms with robust profitability and those with weak profitability
5. CMA: Investment factor (conservative minus agressive) - captures the difference in returns between firms that invest conservatively and those that invest aggressively


In [33]:
# @title 7. Factor exposure

import requests
import zipfile
import io

ff_link = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_5_Factors_2x3_daily_CSV.zip'

# Download the file
response = requests.get(ff_link)

# Check if the request was successful
if response.status_code == 200:
    # Open the zip file from the downloaded content
    with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
        # List the files in the zip archive
        # zf.printdir()
        # Extract and read the CSV file (usually there is only one)
        with zf.open('F-F_Research_Data_5_Factors_2x3_daily.CSV') as file:
            df_ff_factor = pd.read_csv(file, skiprows=3,index_col=0)/100
            df_ff_factor.index = pd.to_datetime(df_ff_factor.index, format='%Y%m%d')

else:
    print("Failed to download the file. Status code:", response.status_code)


# Ensure alignment before performing factor regression
common_index = ret_all.index.intersection(df_ff_factor.index)
etf_factor = pc.factor_exposure_coefficient(ret_all.loc[common_index], df_ff_factor.loc[common_index], all_tickers)
display(etf_factor.style.background_gradient(cmap='Blues', subset=etf_factor.columns[:],axis=None))

# Filter all_port_ret to the common index as well for portfolio factor analysis
common_index_2 = all_port_ret.index.intersection(df_ff_factor.index)
all_port_ret_filtered = all_port_ret.loc[common_index_2]
port_factor = pc.factor_exposure_coefficient(all_port_ret_filtered, df_ff_factor.loc[common_index_2], port_names)
display(port_factor.style.background_gradient(cmap='Blues', subset=port_factor.columns[:],axis=None))

,Mkt,SMB,RMW,HML,CMA
Core Growth,0.63,0.07,0.00,0.00,0.00
Core E100,0.88,0.00,0.00,0.00,0.00
Thematic ESG & Clean Energy,0.91,0.24,-0.12,0.00,0.00
ACWI,0.90,-0.02,0.00,0.06,0.00
Thematic Disruptive Tech,1.03,0.28,-0.59,-0.27,-0.34


,Mkt,RMW,HML,SMB,CMA
P1,0.92,-0.32,-0.12,0.11,-0.11
P2,0.73,-0.12,0.00,0.11,0.00


# Export data to excel

In [34]:
# @title 8. Input file name and export key data to excel

file_path_widget = widgets.Text(
    value='Portfolio_Visualizer_Output.xlsx',
    description='File Name:',
    disabled=False
)

display(file_path_widget) # Display the widget

# Function to save the data to Excel
def save_to_excel(file_path):
    with pd.ExcelWriter(file_path) as writer:
      constituent_summary.to_excel(writer, sheet_name='Constituents summary', index=True)
      yearly_df.to_excel(writer, sheet_name='Constituents calendar year returns', index=True)
      perf_summary.to_excel(writer, sheet_name='Perf summary', index=True)
      port_yearly_ret.to_excel(writer, sheet_name='Calendar year returns', index=True)
      df_ports_alloc_latest.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage).to_excel(writer, sheet_name='Latest alloc', index=True)
      df_ports_alloc_avg.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage).to_excel(writer, sheet_name='Average alloc', index=True)
      assetclass_breakdown_all.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage).to_excel(writer, sheet_name='Asset class exposure', index=True)
      country_breakdown_all.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage).to_excel(writer, sheet_name='Country exposure', index=True)
      sector_breakdown_all.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage).to_excel(writer, sheet_name='Sector exposure', index=True)
      etf_factor.sort_values(by='Mkt', ascending=False).style.background_gradient(cmap='Blues', subset=etf_factor.columns[:],axis=None).to_excel(writer, sheet_name='ETF factor', index=True)
      port_factor.sort_values(by='Mkt', ascending=False).style.background_gradient(cmap='Blues', subset=etf_factor.columns[:],axis=None).to_excel(writer, sheet_name='Portfolio factor', index=True)
      df_forecast.to_excel(writer, sheet_name='Forecast simulation', index=True)

      for n in port_names:
          all_port_wgt[n].to_excel(writer, sheet_name=n+'_hist_wgt', index=True)

      print(f"Excel file saved to: Projects/Projects/{file_path}")

# Create a button to trigger the save function
save_button = widgets.Button(description="Save as Excel")

# Define the button's on_click behavior
def on_save_button_clicked(b):
    save_to_excel(file_path_widget.value)

save_button.on_click(on_save_button_clicked)
display(save_button) # Display the button

Text(value='Portfolio_Visualizer_Output.xlsx', description='File Name:')

Button(description='Save as Excel', style=ButtonStyle())